In [25]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\Joaquín Amat\Documents\GitHub\skforecast


In [42]:
import re
import pytest
import joblib
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import bayesian_search_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold, OneStepAheadFold
from skforecast.preprocessing import RollingFeatures
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.metrics import mean_absolute_scaled_error
import os

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import series
THIS_DIR = Path("C:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast\\skforecast\\model_selection\\tests")
series_item_sales = pd.read_parquet(THIS_DIR/'fixture_multi_series_items_sales.parquet')
series_item_sales = series_item_sales.asfreq('D')
exog_item_sales = pd.DataFrame({'day_of_week': series_item_sales.index.dayofweek}, index = series_item_sales.index)
series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

In [43]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
# from skopt.space import Real
from skforecast.recursive import ForecasterRecursive
from skforecast.model_selection._search import bayesian_search_forecaster
from skforecast.model_selection._split import TimeSeriesFold

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection import y

In [51]:
import os
import re
import sys
import pytest
from pytest import approx
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._search import _bayesian_search_optuna_multiseries
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures
import optuna
from optuna.samplers import TPESampler
from tqdm import tqdm
from functools import partialmethod

import re
import pytest
import os
import sys
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection._search import _bayesian_search_optuna
from skforecast.model_selection._split import TimeSeriesFold
import optuna
from optuna.samplers import TPESampler
from tqdm import tqdm
from functools import partialmethod

In [61]:
def test_results_output_bayesian_search_optuna_window_features_ForecasterAutoregDirect():
    """
    Test output of _bayesian_search_optuna in ForecasterDirect including
    window_features with mocked (mocked done in Skforecast v0.4.3).
    """
    window_features = RollingFeatures(
        stats = ['mean', 'std', 'min', 'max', 'sum', 'median', 'ratio_min_max', 'coef_variation'],
        window_sizes = 3,
    )
    forecaster = ForecasterDirect(
                     regressor = RandomForestRegressor(random_state=123),
                     steps     = 3,
                     lags      = 4,
                     window_features = window_features,
                 )
    n_validation = 12
    y_train = y[:-n_validation]
    cv = TimeSeriesFold(
            steps                 = 3,
            initial_train_size    = len(y_train),
            window_size           = None,
            differentiation       = None,
            refit                 = True,
            fixed_train_size      = True,
            gap                   = 0,
            skip_folds            = None,
            allow_incomplete_fold = True,
            return_all_indexes    = False,
        )
    
    def search_space(trial):
        search_space  = {
            'n_estimators'    : trial.suggest_int('n_estimators', 10, 20),
            'min_samples_leaf': trial.suggest_float('min_samples_leaf', 0.1, 1., log=True),
            'max_features'    : trial.suggest_categorical('max_features', ['log2', 'sqrt']),
            'lags'            : trial.suggest_categorical('lags', [2, 4])
        } 
        
        return search_space

    results = _bayesian_search_optuna(
                  forecaster   = forecaster,
                  y            = y,
                  cv           = cv,
                  search_space = search_space,
                  metric       = 'mean_absolute_error',
                  n_trials     = 10,
                  random_state = 123,
                  return_best  = False,
                  verbose      = False
              )[0]
    
    expected_results = pd.DataFrame(
        {
            'lags': [
                np.array([1, 2, 3, 4]),
                np.array([1, 2]),
                np.array([1, 2]),
                np.array([1, 2, 3, 4]),
                np.array([1, 2]),
                np.array([1, 2, 3, 4]),
                np.array([1, 2]),
                np.array([1, 2]),
                np.array([1, 2]),
                np.array([1, 2]),
            ],
            'params': [
                {
                    'n_estimators': 20,
                    'min_samples_leaf': 0.4839825891759374,
                    'max_features': 'log2',
                },
                {
                    'n_estimators': 15,
                    'min_samples_leaf': 0.41010449151752726,
                    'max_features': 'sqrt',
                },
                {
                    'n_estimators': 14,
                    'min_samples_leaf': 0.782328520465639,
                    'max_features': 'log2',
                },
                {
                    'n_estimators': 17,
                    'min_samples_leaf': 0.21035794225904136,
                    'max_features': 'log2',
                },
                {
                    'n_estimators': 15,
                    'min_samples_leaf': 0.34027307604369605,
                    'max_features': 'sqrt',
                },
                {
                    'n_estimators': 13,
                    'min_samples_leaf': 0.2599119286878713,
                    'max_features': 'log2',
                },
                {
                    'n_estimators': 13,
                    'min_samples_leaf': 0.20142843988664705,
                    'max_features': 'sqrt',
                },
                {
                    'n_estimators': 14,
                    'min_samples_leaf': 0.1147302385573586,
                    'max_features': 'sqrt',
                },
                {
                    'n_estimators': 17,
                    'min_samples_leaf': 0.19325882509735576,
                    'max_features': 'sqrt',
                },
                {
                    'n_estimators': 11,
                    'min_samples_leaf': 0.2714570796881701,
                    'max_features': 'sqrt',
                },
            ],
            'mean_absolute_error': [
                0.2130714159765625,
                0.21370598876262625,
                0.21466582599386722,
                0.21724682621084832,
                0.21740193466522548,
                0.22083884690087485,
                0.22198405983119776,
                0.2246121914126593,
                0.22842351518266302,
                0.22906678546095408,
            ],
            'n_estimators': [20, 15, 14, 17, 15, 13, 13, 14, 17, 11],
            'min_samples_leaf': [
                0.4839825891759374,
                0.41010449151752726,
                0.782328520465639,
                0.21035794225904136,
                0.34027307604369605,
                0.2599119286878713,
                0.20142843988664705,
                0.1147302385573586,
                0.19325882509735576,
                0.2714570796881701,
            ],
            'max_features': [
                'log2',
                'sqrt',
                'log2',
                'log2',
                'sqrt',
                'log2',
                'sqrt',
                'sqrt',
                'sqrt',
                'sqrt',
            ],
        }
    )

    pd.testing.assert_frame_equal(results, expected_results)

test_results_output_bayesian_search_optuna_window_features_ForecasterAutoregDirect()

  0%|          | 0/10 [00:00<?, ?it/s]

In [59]:
window_features = RollingFeatures(
    stats = ['mean', 'std', 'min', 'max', 'sum', 'median', 'ratio_min_max', 'coef_variation'],
    window_sizes = 3,
)
forecaster = ForecasterDirect(
                    regressor = RandomForestRegressor(random_state=123),
                    steps     = 3,
                    lags      = 4,
                    window_features = window_features,
                )
n_validation = 12
y_train = y[:-n_validation]
cv = TimeSeriesFold(
        steps                 = 3,
        initial_train_size    = len(y_train),
        window_size           = None,
        differentiation       = None,
        refit                 = True,
        fixed_train_size      = True,
        gap                   = 0,
        skip_folds            = None,
        allow_incomplete_fold = True,
        return_all_indexes    = False,
    )

def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 10, 20),
        'min_samples_leaf': trial.suggest_float('min_samples_leaf', 0.1, 1., log=True),
        'max_features'    : trial.suggest_categorical('max_features', ['log2', 'sqrt']),
        'lags'            : trial.suggest_categorical('lags', [2, 4])
    } 
    
    return search_space

results = _bayesian_search_optuna(
                forecaster   = forecaster,
                y            = y,
                cv           = cv,
                search_space = search_space,
                metric       = 'mean_absolute_error',
                n_trials     = 10,
                random_state = 123,
                return_best  = False,
                verbose      = False
            )[0]
results

  0%|          | 0/10 [00:00<?, ?it/s]

,lags,params,mean_absolute_error,n_estimators,min_samples_leaf,max_features
0,"[1, 2, 3, 4]","{'n_estimators': 20, 'min_samples_leaf': 0.483...",0.213071,20,0.483983,log2
1,"[1, 2]","{'n_estimators': 15, 'min_samples_leaf': 0.410...",0.213706,15,0.410104,sqrt
2,"[1, 2]","{'n_estimators': 14, 'min_samples_leaf': 0.782...",0.214666,14,0.782329,log2
3,"[1, 2, 3, 4]","{'n_estimators': 17, 'min_samples_leaf': 0.210...",0.217247,17,0.210358,log2
4,"[1, 2]","{'n_estimators': 15, 'min_samples_leaf': 0.340...",0.217402,15,0.340273,sqrt
5,"[1, 2, 3, 4]","{'n_estimators': 13, 'min_samples_leaf': 0.259...",0.220839,13,0.259912,log2
6,"[1, 2]","{'n_estimators': 13, 'min_samples_leaf': 0.201...",0.221984,13,0.201428,sqrt
7,"[1, 2]","{'n_estimators': 14, 'min_samples_leaf': 0.114...",0.224612,14,0.114730,sqrt
8,"[1, 2]","{'n_estimators': 17, 'min_samples_leaf': 0.193...",0.228424,17,0.193259,sqrt
9,"[1, 2]","{'n_estimators': 11, 'min_samples_leaf': 0.271...",0.229067,11,0.271457,sqrt


In [60]:
results.to_dict(orient='list')

{'lags': [array([1, 2, 3, 4]),
  array([1, 2]),
  array([1, 2]),
  array([1, 2, 3, 4]),
  array([1, 2]),
  array([1, 2, 3, 4]),
  array([1, 2]),
  array([1, 2]),
  array([1, 2]),
  array([1, 2])],
 'params': [{'n_estimators': 20,
   'min_samples_leaf': 0.4839825891759374,
   'max_features': 'log2'},
  {'n_estimators': 15,
   'min_samples_leaf': 0.41010449151752726,
   'max_features': 'sqrt'},
  {'n_estimators': 14,
   'min_samples_leaf': 0.782328520465639,
   'max_features': 'log2'},
  {'n_estimators': 17,
   'min_samples_leaf': 0.21035794225904136,
   'max_features': 'log2'},
  {'n_estimators': 15,
   'min_samples_leaf': 0.34027307604369605,
   'max_features': 'sqrt'},
  {'n_estimators': 13,
   'min_samples_leaf': 0.2599119286878713,
   'max_features': 'log2'},
  {'n_estimators': 13,
   'min_samples_leaf': 0.20142843988664705,
   'max_features': 'sqrt'},
  {'n_estimators': 14,
   'min_samples_leaf': 0.1147302385573586,
   'max_features': 'sqrt'},
  {'n_estimators': 17,
   'min_samples

In [ ]:
{
    'lags': [
        np.array([1, 2, 3, 4]),
        np.array([1, 2]),
        np.array([1, 2]),
        np.array([1, 2]),
        np.array([1, 2, 3, 4]),
        np.array([1, 2, 3, 4]),
        np.array([1, 2]),
        np.array([1, 2]),
        np.array([1, 2]),
        np.array([1, 2]),
    ],
    'params': [
        {
            'n_estimators': 20,
            'min_samples_leaf': 0.4839825891759374,
            'max_features': 'log2',
        },
        {
            'n_estimators': 15,
            'min_samples_leaf': 0.41010449151752726,
            'max_features': 'sqrt',
        },
        {
            'n_estimators': 14,
            'min_samples_leaf': 0.782328520465639,
            'max_features': 'log2',
        },
        {
            'n_estimators': 15,
            'min_samples_leaf': 0.34027307604369605,
            'max_features': 'sqrt',
        },
        {
            'n_estimators': 13,
            'min_samples_leaf': 0.2599119286878713,
            'max_features': 'log2',
        },
        {
            'n_estimators': 17,
            'min_samples_leaf': 0.21035794225904136,
            'max_features': 'log2',
        },
        {
            'n_estimators': 11,
            'min_samples_leaf': 0.2714570796881701,
            'max_features': 'sqrt',
        },
        {
            'n_estimators': 13,
            'min_samples_leaf': 0.20142843988664705,
            'max_features': 'sqrt',
        },
        {
            'n_estimators': 17,
            'min_samples_leaf': 0.19325882509735576,
            'max_features': 'sqrt',
        },
        {
            'n_estimators': 14,
            'min_samples_leaf': 0.1147302385573586,
            'max_features': 'sqrt',
        },
    ],
    'mean_absolute_error': [
        0.21252324019730395,
        0.21851156682698414,
        0.22027505037414963,
        0.22530355563144466,
        0.2378358005415178,
        0.2384269598653063,
        0.25241649504238356,
        0.2584381014601676,
        0.26067982740527423,
        0.2697438465540914,
    ],
    'n_estimators': [20, 15, 14, 15, 13, 17, 11, 13, 17, 14],
    'min_samples_leaf': [
        0.4839825891759374,
        0.41010449151752726,
        0.782328520465639,
        0.34027307604369605,
        0.2599119286878713,
        0.21035794225904136,
        0.2714570796881701,
        0.20142843988664705,
        0.19325882509735576,
        0.1147302385573586,
    ],
    'max_features': [
        'log2',
        'sqrt',
        'log2',
        'sqrt',
        'log2',
        'log2',
        'sqrt',
        'sqrt',
        'sqrt',
        'sqrt',
    ],
}
